# सत्र ६ – बहु-चरण पाइपलाइन (योजना → कार्यान्वयन → परिमार्जन)

प्रत्येक चरणका लागि मोडेल चयन गर्न राउटिङ कार्यहरू प्रयोग गरिन्छ, त्यसपछि अन्तिम उत्तरलाई परिमार्जन गरिन्छ।


### व्याख्या: निर्भरता स्थापना  
`foundry-local-sdk` र `openai` स्थापना गर्दछ जुन प्रति चरण च्याट कलहरूको लागि आवश्यक छ। पुन: चलाउन सुरक्षित।


# परिदृश्य
बहु-चरण मोडेल पाइपलाइन जसले: (1) कार्यलाई छुट्टाछुट्टै चरणहरूमा योजना बनाउँछ, (2) प्रत्येक चरणलाई उद्देश्य-आधारित मोडेल चयनको साथ कार्यान्वयन गर्छ, (3) अन्तिम संश्लेषित उत्तरलाई परिमार्जन गर्छ। दक्षता कायम राख्दै गुणस्तर सुरक्षित गर्नका लागि विविध SLM क्षमताहरूको श्रृंखलाबद्धता प्रदर्शन गर्दछ।


In [13]:
!pip install -q foundry-local-sdk openai

### व्याख्या: मुख्य आयातहरू
इन्टेन्ट पत्ता लगाउन regex आयात गरिन्छ, प्रति-अलियास अट्याचमेन्टको लागि Foundry Local manager आयात गरिन्छ, र च्याट पूरा गर्न OpenAI client आयात गरिन्छ।


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### व्याख्या: क्षमता सूची र नियमहरू
क्षमता-सचेत सूची र रेगुलर एक्सप्रेशन नियमहरू परिभाषित गर्दछ, जसले चरण पाठलाई उद्देश्य श्रेणीहरू (कोड, संक्षेप, वर्गीकरण, सामान्य) मा म्याप गर्न प्रयोग गरिन्छ। जब धेरै मोडेलहरूले एउटै उद्देश्यलाई समर्थन गर्छन्, कम प्राथमिकता मानहरूले टक्करमा जित्छ।


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### व्याख्या: उद्देश्य, मोडेल चयन र च्याट सहायक
प्रदान गर्दछ:
- `detect_intent` रेगुलर एक्सप्रेशन आधारित वर्गीकरणको लागि।
- `pick_model` क्षमता + प्राथमिकताका आधारमा उत्कृष्ट उपनाम चयन गर्न।
- `chat` सुविधा प्रदान गर्ने आवरण जसले प्रत्येक उपनामको लागि क्लाइन्ट सिर्जना गर्छ र एक पटकको प्रतिक्रिया फर्काउँछ।


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### व्याख्या: बहु-चरण पाइपलाइन कार्य
पाइपलाइन लागू गर्दछ: योजना → चरणहरू पार्स गर्नुहोस् → प्रत्येकलाई उद्देश्य-आधारित रुटिङसँग कार्यान्वयन गर्नुहोस् → संयुक्त नतिजाहरू परिष्कृत गर्नुहोस्। निरीक्षण वा मूल्याङ्कनको लागि संरचित dict फर्काउँछ।


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### व्याख्या: उदाहरण कार्य चलाउनुहोस्
रिफ्याक्टर-केन्द्रित कार्यमा पूर्ण पाइपलाइन चलाउँछ जसले मिश्रित उद्देश्यहरू (कोड + संक्षेप) देखाउँछ। परिणाम डिक्टले कच्चा योजना, प्रत्येक चरणको आउटपुटहरू चयन गरिएको मोडेल उपनामहरूसँग, र अन्तिम संश्लेषित उत्तर देखाउँछ।


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### व्याख्या: परिणाम वस्तु प्रदर्शन गर्नुहोस्  
द्रुत निरीक्षण वा पछिल्लो मूल्याङ्कनको लागि संरचित पाइपलाइन आउटपुट देखाउँछ (जस्तै, चरणको गुणस्तर वा सुधारको प्रभावकारिता मापन गर्न)।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी यथार्थताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुनेछैनौं।
